In [9]:
import numpy as np
from math import exp, sqrt, log
from scipy.stats import norm

## Black-Scholes Partial Differential Equation (PDE)

The Black-Scholes PDE is given by:

$$
\frac{\partial V}{\partial t} + \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + r S \frac{\partial V}{\partial S} - rV = 0
$$

where:
- $V(S,t)$ is the price of the option as a function of the asset price $S$ and time $t$.
- $\sigma$ is the volatility.
- $r$ is the risk-free interest rate.

This PDE is derived from no-arbitrage arguments and the construction of a risk-free portfolio, and its solution yields the closed-form Black-Scholes formula for European call (and put) options.

## American (US) Call and Put Options

American options differ from European options in that they allow early exercise. Unlike the Black-Scholes closed-form solution for European options, American options generally do not have a closed-form solution and are typically priced using numerical methods (e.g., binomial trees, finite difference methods, or Monte Carlo simulations).

- For an American call on a non-dividend paying asset, early exercise is generally suboptimal, so its price is effectively the same as the European call.
- For an American put, early exercise may be optimal, resulting in a premium over the European put price.

## Black-Scholes Formula

The Black-Scholes formula for a European call option is given by:

$$
d_1 = \frac{\ln\left(\frac{S}{K}\right) + \left(r + \frac{1}{2}\sigma^2\right)T}{\sigma\sqrt{T}}
$$

$$
d_2 = d_1 - \sigma\sqrt{T}
$$

The option price is computed as:

$$
C = S \cdot N(d_1) - K \cdot e^{-rT} \cdot N(d_2)
$$

Where:  
- $S$ is the current asset price.  
- $K$ is the strike price.  
- $T$ is the time to maturity in years.  
- $r$ is the risk-free interest rate.  
- $\sigma$ is the volatility.  
- $N(\cdot)$ denotes the cumulative normal distribution function.

In [10]:
# The following implementation of the Black-Scholes formula is correct:
# d1 = (log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*sqrt(T))
# d2 = d1 - sigma*sqrt(T)
def black_scholes_call(S, K, T, r, sigma):
    """
    S: asset price
    K: strike price
    T: time to maturity (years)
    r: risk-free interest rate
    sigma: volatility
    """
    d1 = (log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    return S * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)

## Monte Carlo Simulation for European Call Option

The Monte Carlo simulation estimates the price of a European call option by simulating the terminal asset price using geometric Brownian motion. The model for the terminal price is:

$$
S_T = S \cdot \exp\left[\left(r - \frac{1}{2}\sigma^2\right)T + \sigma\sqrt{T}\,Z\right]
$$

where \(Z\) is a standard normal random variable (\(Z \sim N(0,1)\)). The payoff of the call option is given by:

$$
\text{payoff} = \max(S_T - K, \, 0)
$$

The option price is then estimated as the discounted average payoff:

$$
C = e^{-rT} \cdot \mathbb{E}[\text{payoff}]
$$

**Clarifications:**
- **Z**: A standard normal random variable used in the simulation.
- **payoff**: The value of the option at expiration, calculated as $\max(S_T - K, 0)$.
- **C**: The estimated call option price after discounting the average payoff.

Where:  
- $S$ is the current asset price.  
- $K$ is the strike price.  
- $T$ is the time to maturity in years.  
- $r$ is the risk-free interest rate.  
- $\sigma$ is the volatility.  
- $N(\cdot)$ denotes the cumulative normal distribution function.


In [11]:
# The following implementation of the Monte Carlo simulation for a European call option is correct:
# - Uses np.random.seed for reproducibility.
# - Simulates the terminal asset price with geometric Brownian motion.
# - Computes the discounted average payoff.
def monte_carlo_call_price(S, K, T, r, sigma, simulations=10000):
    """
    S: asset price
    K: strike price
    T: time to maturity (years)
    r: risk-free interest rate
    sigma: volatility
    """
    np.random.seed(42)  # for reproducibility
    Z = np.random.standard_normal(simulations)
    ST = S * np.exp((r - 0.5 * sigma**2) * T + sigma * sqrt(T) * Z)
    payoff = np.maximum(ST - K, 0)
    return exp(-r * T) * np.mean(payoff)

In [14]:
def main():
    S = 100      # asset price
    K = 100      # strike price
    T = 10      # time to maturity (years)
    r = 0.05     # risk-free interest rate
    sigma = 0.2  # volatility

    bs_price = black_scholes_call(S, K, T, r, sigma)
    mc_price = monte_carlo_call_price(S, K, T, r, sigma)
    print("Black-Scholes Call Price:", bs_price)
    print("Monte Carlo Call Price:", mc_price)

if __name__ == "__main__":
    main()

Black-Scholes Call Price: 45.19297367898936
Monte Carlo Call Price: 45.25215386645332


Significance of this:

The close agreement between the Black-Scholes call price (45.19) and the Monte Carlo call price (45.25) indicates that the Monte Carlo simulation is accurately capturing the theoretical pricing model. It validates both methods, despite their different approaches, with the small discrepancy likely due to simulation randomness and numerical approximations